In [1]:
"""
!python -m pip install -U setuptools pip
!python -m pip install -U easydict
!python -m pip install -U k3d
!python -m pip install -U tqdm
!jupyter nbextension install --py --user k3d
!jupyter nbextension enable --py --user k3d
!python -m pip install -U cupy-cuda11x numba
"""

!nvidia-smi

Tue Feb 13 03:52:24 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.223.02   Driver Version: 470.223.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 58%   33C    P8    28W / 350W |    697MiB / 24265MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:21:00.0 Off |                  N/A |
| 33%   

In [2]:
import math
import os
import cupy as cp
from numba import cuda
import torch
from matplotlib import pyplot as plt
import k3d
from easydict import EasyDict as edict

%matplotlib widget

device_num =0
cp.cuda.Device(device_num).use()
cuda.select_device(device_num)
torch.cuda.set_device(device_num)
solver = 'unet'

ckpt_file = './checkpoint/3d_buckling/ckpt.pth'#'viscosity_learning_3d/checkpoint/0608/model_epoch11.pth'#'viscosity_learning_3d/checkpoint/1012orig/model_epoch192.pth'

In [3]:
# P2G functions
@cuda.jit
def p2g_particle(px, pm, pv, pca, gm, gv, bound_size, bound_min, gres, grid_bias, cell_size, axis):
    P = cuda.grid(1)
    if P >= px.shape[0]:
        return

    m = pm[P]
    x = cuda.local.array(3, dtype=cp.float32)
    v = cuda.local.array(3, dtype=cp.float32)
    gi = cuda.local.array(3, dtype=cp.int64)    # grid index
    gx = cuda.local.array(3, dtype=cp.float32)  # grid position
    disp = cuda.local.array(3, dtype=cp.float32)
    w = cuda.local.array(3, dtype=cp.float32)   # weight |gx - x| / cell_size
    for d in range(3):
        x[d] = px[P,d]
        v[d] = pv[P,d]
        gi[d] = math.floor((x[d] - bound_min[d]) / cell_size[d] - grid_bias[d])
        gx[d] = (gi[d] + grid_bias[d]) * cell_size[d] + bound_min[d]
        disp[d] = gx[d] - x[d]
        w[d] = abs(disp[d]) / cell_size[d]
    
    for ix in [0,1]:
        for iy in [0,1]:
            for iz in [0,1]:
                gix = max(0, min(gres[0]-1, gi[0] + ix))
                giy = max(0, min(gres[1]-1, gi[1] + iy))
                giz = max(0, min(gres[2]-1, gi[2] + iz))
                wx = ix + ((-1)**ix) * (1 - w[0])
                wy = iy + ((-1)**iy) * (1 - w[1])
                wz = iz + ((-1)**iz) * (1 - w[2])
                cv = (
                    (disp[0] + ix*cell_size[0])*pca[P,0]
                    + (disp[1] + iy*cell_size[1])*pca[P,1]
                    + (disp[2] + iz*cell_size[2])*pca[P,2]
                )
                weight = wx * wy * wz
                cuda.atomic.add(gm, (gix, giy, giz), weight * m)
                cuda.atomic.add(gv, (gix, giy, giz), weight * m * (v[axis] + cv))

@cuda.jit
def p2g_grid(gm, gv):
    x,y,z = cuda.grid(3)
    if x >= gm.shape[0] or y >= gm.shape[1] or z >= gm.shape[2]:
        return
    
    m = gm[x,y,z]
    if m > 0:
        gv[x,y,z] = gv[x,y,z] / m

def p2g(p, g):
    P = p.num_particles
    THREAD1 = 512
    THREAD3 = (8,8,8)
    
    particle_blocks = (P - 1) // THREAD1 + 1
    threads = cp.array(THREAD3, dtype=cp.int64)
    gx_blocks = (*((g.x.resolution - 1) // threads + 1).get(),)
    gy_blocks = (*((g.y.resolution - 1) // threads + 1).get(),)
    gz_blocks = (*((g.z.resolution - 1) // threads + 1).get(),)

    p2g_particle[particle_blocks, THREAD1](p.x, p.m, p.v, p.cx, g.x.m, g.x.v, g.bound_size, g.bound_min, g.resolution, g.x.bias, g.cell_size, 0)
    p2g_particle[particle_blocks, THREAD1](p.x, p.m, p.v, p.cy, g.y.m, g.y.v, g.bound_size, g.bound_min, g.resolution, g.y.bias, g.cell_size, 1)
    p2g_particle[particle_blocks, THREAD1](p.x, p.m, p.v, p.cz, g.z.m, g.z.v, g.bound_size, g.bound_min, g.resolution, g.z.bias, g.cell_size, 2)
    
    cuda.synchronize()
    
    p2g_grid[gx_blocks, THREAD3](g.x.m, g.x.v)
    p2g_grid[gy_blocks, THREAD3](g.y.m, g.y.v)
    p2g_grid[gz_blocks, THREAD3](g.z.m, g.z.v)


In [4]:
# G2P functions
@cuda.jit
def g2p_particle(bound_size, bound_min, gres, grid_bias, cell_size, axis, px, pm, pv, pca, gv):
    P = cuda.grid(1)
    if P >= px.shape[0]:
        return

    x = cuda.local.array(3, dtype=cp.float32)
    gi = cuda.local.array(3, dtype=cp.int64)    # grid index
    gx = cuda.local.array(3, dtype=cp.float32)  # grid position
    w = cuda.local.array(3, dtype=cp.float32)   # weight |gx - x| / cell_size
    for d in range(3):
        x[d] = px[P,d]
        gi[d] = math.floor((x[d] - bound_min[d]) / cell_size[d] - grid_bias[d])
        gx[d] = (gi[d] + grid_bias[d]) * cell_size[d] + bound_min[d]
        w[d] = abs(gx[d] - x[d]) / cell_size[d]
        pca[P,d] = 0
    
    pv[P,axis] = 0.0
    for ix in [0,1]:
        for iy in [0,1]:
            for iz in [0,1]:
                gix = max(0, min(gres[0]-1, gi[0] + ix))
                giy = max(0, min(gres[1]-1, gi[1] + iy))
                giz = max(0, min(gres[2]-1, gi[2] + iz))
                wx = 1 - ix + (2*ix - 1)*w[0]  # ix=0 --> 1-w; ix=1 --> w
                wy = 1 - iy + (2*iy - 1)*w[1]
                wz = 1 - iz + (2*iz - 1)*w[2]
                weight = wx * wy * wz

                pv[P,axis] += weight * gv[gix, giy, giz]
                pca[P,0] += (2*ix-1) * wy * wz * gv[gix, giy, giz] / cell_size[0]
                pca[P,1] += wx * (2*iy-1) * wz * gv[gix, giy, giz] / cell_size[1]
                pca[P,2] += wx * wy * (2*iz-1) * gv[gix, giy, giz] / cell_size[2]

def g2p(p, g):
    P = p.num_particles
    BLOCK1 = 512
    
    particle_blocks = (P - 1) // BLOCK1 + 1

    g2p_particle[particle_blocks, BLOCK1](g.bound_size, g.bound_min, g.resolution, g.x.bias, g.cell_size, 0, p.x, p.m, p.v, p.cx, g.x.v)
    g2p_particle[particle_blocks, BLOCK1](g.bound_size, g.bound_min, g.resolution, g.y.bias, g.cell_size, 1, p.x, p.m, p.v, p.cy, g.y.v)
    g2p_particle[particle_blocks, BLOCK1](g.bound_size, g.bound_min, g.resolution, g.z.bias, g.cell_size, 2, p.x, p.m, p.v, p.cz, g.z.v)

In [5]:
# Fluid levelset functions
@cuda.jit(device=True)
def norm(v):
    n = 0.0
    for d in range(3):
        n += v[d] * v[d]
    return n ** 0.5

@cuda.jit
def compute_fls_kernel(px, phi, bound_size, bound_min, cell_size, r, gres):
    P = cuda.grid(1)
    if P >= px.shape[0]:
        return
    
    x = cuda.local.array(3, dtype=cp.float32)
    gi = cuda.local.array(3, dtype=cp.int64)    # grid index
    gx = cuda.local.array(3, dtype=cp.float32)  # grid position
    for d in range(3):
        x[d] = px[P,d]
        gi[d] = math.floor((x[d] - bound_min[d]) / cell_size[d])
        gx[d] = (gi[d] + 0.5) * cell_size[d] + bound_min[d]
    
    gii = cuda.local.array(3, dtype=cp.int64)
    gip = cuda.local.array(3, dtype=cp.float32)
    for dx in range(-2,3):
        for dy in range(-2,3):
            for dz in range(-2,3):
                gii[0] = max(0, min(gres[0]-1, gi[0] + dx))
                gii[1] = max(0, min(gres[1]-1, gi[1] + dy))
                gii[2] = max(0, min(gres[2]-1, gi[2] + dz))

                for d in range(3):
                    gip[d] = (gii[d] + 0.5) * cell_size[d] + bound_min[d] - x[d]

                dist = norm(gip) - r
                cuda.atomic.min(phi, (gii[0], gii[1], gii[2]), dist)

def compute_fluid_levelset(p, ls, gdx):
    P = p.num_particles
    BLOCK1 = 512
    particle_blocks = (P - 1) // BLOCK1 + 1
    
    r = gdx * 0.5 * math.sqrt(3.0) * 1.02
    ls.phi[:] = gdx * 3
    compute_fls_kernel[particle_blocks, BLOCK1](p.x, ls.phi, ls.bound_size, ls.bound_min, ls.cell_size, r, ls.resolution)

In [6]:
# Boundary conditions functions
@cuda.jit
def boundary_condition_x(gvx, gvy, gvz, gmy, gmz, sphi, sv, dx, dv):
    x, y, z = cuda.grid(3)
    if x == 0 or x >= dv.shape[0] - 1 or y == 0 or y >= dv.shape[1]-1 or z == 0 or z >= dv.shape[2]-1:
        # ignore boundary cells
        dv[x,y,z] = 0.0
        return
    
    ndist = sphi[2*x, 2*y+1, 2*z+1] / dx # normalized_distance
    if ndist >= 1:
        dv[x,y,z] = 0.0
        return
    
    vx = gvx[x,y,z]
    vy = 0.0
    vz = 0.0
    my = 0.0
    mz = 0.0
    for ix in range(2):
        for iy in range(2):
                gix = x - ix
                giy = y + iy
                giz = z + iy
                my += gmy[gix,giy,z]
                vy += gvy[gix,giy,z] * gmy[gix,giy,z]
                mz += gmz[gix,y,giz]
                vz += gvz[gix,y,giz] * gmz[gix,y,giz]
    vy /= my
    vz /= mz
    
    vx -= sv[2*x, 2*y+1, 2*z+1, 0]
    vy -= sv[2*x, 2*y+1, 2*z+1, 1]
    vz -= sv[2*x, 2*y+1, 2*z+1, 2]
    
    # solid normal
    snx = sphi[2*x+1, 2*y+1, 2*z+1] - sphi[2*x-1,2*y+1, 2*z+1]
    sny = sphi[2*x, 2*y+2, 2*z+1] - sphi[2*x, 2*y, 2*z+1]
    snz = sphi[2*x, 2*y+1, 2*z+2] - sphi[2*x, 2*y+1, 2*z]
    sn_inv = 1.0 / (snx**2 + sny**2 + snz**2)
    
    # x component of proj(v, sn)
    gvx_sn = min(0, snx*vx + sny*vy + snz*vz) * snx * sn_inv
    
    dv[x,y,z] = -gvx_sn * (1.0 - ndist)
    # dv[x,y,z] = (ndist - 1.0) * gvx[x,y,z]

@cuda.jit
def boundary_condition_y(gvx, gvy, gvz, gmx, gmz, sphi, sv, dx, dv):
    x, y, z = cuda.grid(3)
    if x == 0 or x >= dv.shape[0] - 1 or y == 0 or y >= dv.shape[1]-1 or z == 0 or z >= dv.shape[2]-1:
        # ignore boundary cells
        dv[x,y,z] = 0.0
        return
    
    ndist = sphi[2*x+1, 2*y, 2*z+1] / dx # normalized_distance
    if ndist >= 1:
        dv[x,y,z] = 0.0
        return
    
    vx = 0.0
    vy = gvy[x,y,z]
    vz = 0.0
    mx = 0.0
    mz = 0.0
    for iy in range(2):
        for iz in range(2):
                gix = x + iz
                giy = y - iy
                giz = z + iz
                mx += gmx[gix,giy,z]
                vx += gvx[gix,giy,z] * gmx[gix,giy,z]
                mz += gmz[x,giy,giz]
                vz += gvz[x,giy,giz] * gmz[x,giy,giz]
    vx /= mx
    vz /= mz
    
    vx -= sv[2*x+1, 2*y, 2*z+1, 0]
    vy -= sv[2*x+1, 2*y, 2*z+1, 1]
    vz -= sv[2*x+1, 2*y, 2*z+1, 2]
    
    # solid normal
    snx = sphi[2*x+2, 2*y, 2*z+1] - sphi[2*x,2*y, 2*z+1]
    sny = sphi[2*x+1, 2*y+1, 2*z+1] - sphi[2*x+1, 2*y-1, 2*z+1]
    snz = sphi[2*x+1, 2*y, 2*z+2] - sphi[2*x+1, 2*y, 2*z]
    sn_inv = 1.0 / (snx**2 + sny**2 + snz**2)
    
    # y component of proj(v, sn)
    gvy_sn = min(0, snx*vx + sny*vy + snz*vz) * sny * sn_inv
    
    dv[x,y,z] = -gvy_sn * (1.0 - ndist)
    # dv[x,y,z] = (ndist - 1.0) * gvy[x,y,z]

@cuda.jit
def boundary_condition_z(gvx, gvy, gvz, gmx, gmy, sphi, sv, dx, dv):
    x, y, z = cuda.grid(3)
    if x == 0 or x >= dv.shape[0] - 1 or y == 0 or y >= dv.shape[1]-1 or z == 0 or z >= dv.shape[2]-1:
        # ignore boundary cells
        dv[x,y,z] = 0.0
        return
    
    ndist = sphi[2*x+1, 2*y+1, 2*z] / dx # normalized_distance
    if ndist >= 1:
        dv[x,y,z] = 0.0
        return
    
    vx = 0.0
    vy = 0.0
    vz = gvz[x,y,z]
    mx = 0.0
    my = 0.0
    for iz in range(2):
        for ix in range(2):
                gix = x + ix
                giy = y + ix
                giz = z - iz
                mx += gmx[gix,y,giz]
                vx += gvx[gix,y,giz] * gmx[gix,y, giz]
                my += gmy[x,giy,giz]
                vy += gvy[x,giy,giz] * gmy[x,giy,giz]
    vx /= mx
    vy /= my
    
    vx -= sv[2*x+1, 2*y+1, 2*z, 0]
    vy -= sv[2*x+1, 2*y+1, 2*z, 1]
    vz -= sv[2*x+1, 2*y+1, 2*z, 2]
    
    # solid normal
    snx = sphi[2*x+2, 2*y+1, 2*z] - sphi[2*x, 2*y+1, 2*z]
    sny = sphi[2*x+1, 2*y+2, 2*z] - sphi[2*x+1, 2*y, 2*z]
    snz = sphi[2*x+1, 2*y+1, 2*z+1] - sphi[2*x+1, 2*y+1, 2*z-1]
    sn_inv = 1.0 / (snx**2 + sny**2 + snz**2)
    
    # y component of proj(v, sn)
    gvz_sn = min(0, snx*vx + sny*vy + snz*vz) * snz * sn_inv
    
    dv[x,y,z] = -gvz_sn * (1.0 - ndist)
    # dv[x,y,z] = (ndist - 1.0) * gvz[x,y,z]

def apply_boundary_condition(g, solid, dx):
    THREAD3 = (8, 8, 8)
    threads = cp.array(THREAD3, dtype=cp.int64)
    gx_blocks = (*((g.x.resolution - 1) // threads + 1).get(),)
    gy_blocks = (*((g.y.resolution - 1) // threads + 1).get(),)
    gz_blocks = (*((g.z.resolution - 1) // threads + 1).get(),)
    
    boundary_condition_x[gx_blocks, THREAD3](g.x.v, g.y.v, g.z.v, g.y.m, g.z.m, solid.phi, solid.v, dx, g.x.dv)
    boundary_condition_y[gy_blocks, THREAD3](g.x.v, g.y.v, g.z.v, g.x.m, g.z.m, solid.phi, solid.v, dx, g.y.dv)
    boundary_condition_z[gz_blocks, THREAD3](g.x.v, g.y.v, g.z.v, g.x.m, g.y.m, solid.phi, solid.v, dx, g.z.dv)
    
    cuda.synchronize()
    
    g.x.v += g.x.dv
    g.y.v += g.y.dv
    g.z.v += g.z.dv

In [7]:
# Fluid volume functions
@cuda.jit
def compute_fluid_volume_kernel(bound_min, cell_size, gres, px, pvol, gvol):
    P = cuda.grid(1)
    if P >= px.shape[0]:
        return
    
    x = cuda.local.array(3, dtype=cp.float32)
    gi = cuda.local.array(3, dtype=cp.int64)    # grid index
    gx = cuda.local.array(3, dtype=cp.float32)  # grid position
    w = cuda.local.array(3, dtype=cp.float32)   # weight |gx - x| / cell_size
    for d in range(3):
        x[d] = px[P,d]
        gi[d] = math.floor((x[d] - bound_min[d]) / cell_size[d])
        gx[d] = gi[d] * cell_size[d] + bound_min[d]
        w[d] = abs(gx[d] - x[d]) / cell_size[d]
    
    for ix in [0,1]:
        for iy in [0,1]:
            for iz in [0,1]:
                gix = max(0, min(gres[0]-1, gi[0] + ix))
                giy = max(0, min(gres[1]-1, gi[1] + iy))
                giz = max(0, min(gres[2]-1, gi[2] + iz))
                wx = ix + ((-1)**ix) * (1 - w[0])
                wy = iy + ((-1)**iy) * (1 - w[1])
                wz = iz + ((-1)**iz) * (1 - w[2])
                weight = wx * wy * wz
                cuda.atomic.add(gvol, (gix, giy, giz), weight * pvol)
@cuda.jit
def constrain_fluid_volume_kernel(gvol, cell_vol):
    x, y, z = cuda.grid(3)
    if x >= gvol.shape[0] or y >= gvol.shape[1] or z >= gvol.shape[2]:
        return
    gvol[x,y,z] = min(gvol[x,y,z], cell_vol)
    
def compute_fluid_volume(p, fv, pvol):
    P = p.num_particles
    BLOCK1 = 512
    particle_blocks = (P - 1) // BLOCK1 + 1

    fv.vol[:] = 0.0
    compute_fluid_volume_kernel[particle_blocks, BLOCK1](fv.bound_min, fv.cell_size, fv.resolution, p.x, pvol, fv.vol)
    
    THREAD3 = (8, 8, 8)
    threads = cp.array(THREAD3, dtype=cp.int64)
    grid_blocks = (*((fv.resolution - 1) // threads + 1).get(),)
    
    cell_vol = cp.prod(fv.cell_size).item()
    cuda.synchronize()
    constrain_fluid_volume_kernel[grid_blocks, THREAD3](fv.vol, cell_vol)
    

In [8]:
# Extrapolation functions
@cuda.jit
def extrapolate_kernel(v, valid, new_v, new_valid):
    x, y, z = cuda.grid(3)
    if x == 0 or x >= v.shape[0] - 1 or y == 0 or y >= v.shape[1]-1 or z == 0 or z >= v.shape[2]-1:
        # ignore boundary cells
        return
    
    val = 0.0
    count = 0
    if valid[x,y,z]:
        return
    if valid[x+1,y,z]:
        val += v[x+1,y,z]
        count += 1
    if valid[x-1,y,z]:
        val += v[x-1,y,z]
        count += 1
    if valid[x,y+1,z]:
        val += v[x,y+1,z]
        count += 1
    if valid[x,y-1,z]:
        val += v[x,y-1,z]
        count += 1
    if valid[x,y,z+1]:
        val += v[x,y,z+1]
        count += 1
    if valid[x,y,z-1]:
        val += v[x,y,z-1]
        count += 1
    if count > 0:
        new_v[x,y,z] = val / count
        new_valid[x,y,z] = True
        
def extrapolate(gres, num_iter, vx, vy, vz, mx, my, mz):
    THREAD3 = (8, 8, 8)
    threads = cp.array(THREAD3, dtype=cp.int64)
    gx_blocks = (*(((gres + cp.array([1,0,0], dtype=cp.int64)) - 1) // threads + 1).get(),)
    gy_blocks = (*(((gres + cp.array([0,1,0], dtype=cp.int64)) - 1) // threads + 1).get(),)
    gz_blocks = (*(((gres + cp.array([0,0,1], dtype=cp.int64)) - 1) // threads + 1).get(),)
    
    vx_valid = mx > 0
    vy_valid = my > 0
    vz_valid = mz > 0
    
    new_vx = vx.copy()
    new_vy = vy.copy()
    new_vz = vz.copy()
    
    new_vx_valid = vx_valid.copy()
    new_vy_valid = vy_valid.copy()
    new_vz_valid = vz_valid.copy()
    
    for _ in range(num_iter):
        extrapolate_kernel[gx_blocks, THREAD3](vx, vx_valid, new_vx, new_vx_valid)
        extrapolate_kernel[gy_blocks, THREAD3](vy, vy_valid, new_vy, new_vy_valid)
        extrapolate_kernel[gx_blocks, THREAD3](vz, vz_valid, new_vz, new_vz_valid)
        
        vx[:] = new_vx
        vy[:] = new_vy
        vz[:] = new_vz
        
        vx_valid[:] = new_vx_valid
        vy_valid[:] = new_vy_valid
        vz_valid[:] = new_vz_valid

In [9]:
# Import solvers
from solver.CGSolverBuffer import CGSolverBuffer
from solver.PressureCGSolver3D import PressureCGSolver3D
from solver.DensityCGSolver3D import DensityCGSolver3D
from solver.ViscosityCGSolver3D import ViscosityCGSolver3D
 
import solver.sdf3D as sdf
!nvidia-smi

Tue Feb 13 03:52:27 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.223.02   Driver Version: 470.223.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 58%   38C    P2   107W / 350W |   1370MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:21:00.0 Off |                  N/A |
| 33%   

# buckling scene construction

In [10]:
D = 3
BOUND_MIN = cp.array([-0.3, 0, -0.3], dtype=cp.float32)
BOUND_SIZE = cp.array([0.6, 1.0, 0.6], dtype=cp.float32)

GDX = 0.0125
PDX = 0.00625
GRES = (BOUND_SIZE / GDX).astype(cp.int64)

RHO = 1000
MU = 1000 * 1e-3
DT = 1/300

def add_box(center, size, dx, _filter=None):
    dim = center.shape[0]
    half_size = size * 0.5
    box_min = center - half_size
    grid_dim = (size / dx).astype(cp.int64)
    
    grid_a = [cp.arange(res_a) for res_a in grid_dim]
    grid_idx = cp.stack(cp.meshgrid(*grid_a), axis=-1).astype(cp.float32)
    grid_pos = box_min + size*((grid_idx + 0.5) / grid_dim)
    
    particle_pos = grid_pos.reshape(-1, dim)
    if _filter is not None:
        particle_pos = particle_pos[_filter(particle_pos)]
    particle_pos += cp.random.randn(*particle_pos.shape) * dx * 0.3
    return particle_pos


rb_map={}
rb_d=cp.asarray([])

rb_d, rb_map = sdf.generate_rb(rb_d, rb_map, 'cube', ['box',0.5,0.8,0.5], flip=True, center=[0, 0.5, 0], axis=[0, 1, 0], angle=0) #

obs_height=0.7

rb_d, rb_map = sdf.generate_rb(rb_d, rb_map, 'cube1', ['box',0.67,0.1,1.0], flip=False, center=[-0.34,obs_height,0], axis=[0, 0, 1], angle=-45)#//hole
rb_d, rb_map = sdf.generate_rb(rb_d, rb_map, 'cube2', ['box',0.67,0.1,1.0], flip=False, center=[0.34,obs_height,0], axis=[0, 0, 1], angle=45)#//hole
rb_d, rb_map = sdf.generate_rb(rb_d, rb_map, 'cube3', ['box',1.0,0.1,0.7], flip=False, center=[0,obs_height,-0.3], axis=[1, 0, 0], angle=45)#//hole
rb_d, rb_map = sdf.generate_rb(rb_d, rb_map, 'cube4', ['box',1.0,0.1,0.7],flip=False, center=[0,obs_height,0.3], axis=[1, 0, 0], angle=-45)#//hole


def oob_filter(px):
    sd = cp.zeros(px.shape[:-1])
    vel = cp.zeros_like(px)
    sdf.evaluate(rb_d, sd, vel, px)
    
    return sd >= 0

PX = add_box(cp.array([0.0, 0.65, 0.0]), cp.array([0.3, 0.3, 0.3]), dx=PDX, _filter=oob_filter)#hole4

PN = PX.shape[0]
v_para=cp.zeros((PN,D))
v_para[:,0] = -2

particle = edict({
    'num_particles': PN,
    'x': PX,
    'm': cp.ones(PN) * RHO * (PDX ** D),
    'v': cp.zeros((PN, D)),
    'cx': cp.zeros((PN, D)),
    'cy': cp.zeros((PN, D)),
    'cz': cp.zeros((PN, D)),
    'vol': PDX ** D,
})


grid = edict({
    'resolution': GRES,
    'bound_size': BOUND_SIZE,
    'bound_min': BOUND_MIN,
    'cell_size': BOUND_SIZE / GRES,
    'x': {
        'resolution': GRES + cp.array([1, 0, 0], dtype=cp.int64),
        'bias': cp.array([0, 0.5, 0.5], dtype=cp.float32),
        'm': cp.zeros((GRES + cp.array([1, 0, 0], dtype=cp.int64)).get(), dtype=cp.float32),
        'v': cp.zeros((GRES + cp.array([1, 0, 0], dtype=cp.int64)).get(), dtype=cp.float32),
        'dv': cp.zeros((GRES + cp.array([1, 0, 0], dtype=cp.int64)).get(), dtype=cp.float32),
    },
    'y': {
        'resolution': GRES + cp.array([0, 1, 0], dtype=cp.int64),
        'bias': cp.array([0.5, 0, 0.5], dtype=cp.float32),
        'm': cp.zeros((GRES + cp.array([0, 1, 0], dtype=cp.int64)).get(), dtype=cp.float32),
        'v': cp.zeros((GRES + cp.array([0, 1, 0], dtype=cp.int64)).get(), dtype=cp.float32),
        'dv': cp.zeros((GRES + cp.array([0, 1, 0], dtype=cp.int64)).get(), dtype=cp.float32),
    },
    'z': {
        'resolution': GRES + cp.array([0, 0, 1], dtype=cp.int64),
        'bias': cp.array([0.5, 0.5, 0], dtype=cp.float32),
        'm': cp.zeros((GRES + cp.array([0, 0, 1], dtype=cp.int64)).get(), dtype=cp.float32),
        'v': cp.zeros((GRES + cp.array([0, 0, 1], dtype=cp.int64)).get(), dtype=cp.float32),
        'dv': cp.zeros((GRES + cp.array([0, 0, 1], dtype=cp.int64)).get(), dtype=cp.float32),
    },
})

SOL_GRES = 2*GRES
SOL_ARRES = SOL_GRES + 1
solid_levelset = edict({
    'resolution': SOL_ARRES,
    'bound_size': BOUND_SIZE,
    'bound_min': BOUND_MIN,
    'cell_size': BOUND_SIZE / SOL_GRES,
    'bias': cp.array([0,0,0], dtype=cp.float32),
    'phi': cp.zeros(SOL_ARRES.get()),
    'pos': cp.zeros((*SOL_ARRES.get(), D)),
    'v': cp.zeros((*SOL_ARRES.get(), D)),
})

FL_GRES = GRES.copy()
fluid_levelset = edict({
    'resolution': FL_GRES,
    'bound_size': BOUND_SIZE,
    'bound_min': BOUND_MIN,
    'cell_size': BOUND_SIZE / FL_GRES,
    'phi': cp.zeros(FL_GRES.get()),
})

VOL_GRES = 2*GRES
VOL_ARRES = VOL_GRES + 1
fluid_volume = edict({
    'resolution': VOL_ARRES,
    'bound_size': BOUND_SIZE,
    'bound_min': BOUND_MIN,
    'cell_size': BOUND_SIZE / VOL_GRES,
    'vol': cp.zeros(VOL_ARRES.get()),
})

CGBuf = CGSolverBuffer(GRES)
PressureSolver = PressureCGSolver3D(CGBuf, GRES, GDX)
DensitySolver = DensityCGSolver3D(CGBuf, GRES, BOUND_MIN, BOUND_SIZE)
ViscositySolver = ViscosityCGSolver3D(GRES, BOUND_SIZE)

def get_grid_pos(gres, bound_min, bound_size, cell_size, bias):
    grid_a = [cp.arange(res_a) for res_a in gres]
    grid_idx = cp.stack(cp.meshgrid(*grid_a, indexing='ij'), axis=-1).astype(cp.float32)
    grid_pos = bound_min + ((grid_idx + bias)*cell_size)
    return grid_pos

solid_levelset.pos[:] = get_grid_pos(solid_levelset.resolution, solid_levelset.bound_min, solid_levelset.bound_size, solid_levelset.cell_size, solid_levelset.bias)

# Debugging rigid bodies
sdf.evaluate(rb_d, solid_levelset.phi, solid_levelset.v, solid_levelset.pos)


K3D_BOUND = []
K3D_BOUND.append(BOUND_MIN[[0,2,1]].get())
K3D_BOUND.append((BOUND_MIN+BOUND_SIZE)[[0,2,1]].get())

sphi_np = solid_levelset.phi.astype(cp.float32).get().transpose(1,2,0)

plot = k3d.plot(grid=[*K3D_BOUND[0], *K3D_BOUND[1]], grid_auto_fit=False)
plot += k3d.marching_cubes(sphi_np, level=0, opacity=0.2, color=0xFF0000, bounds=[K3D_BOUND[i%2][i//2] for i in range(6)])
plot += k3d.points(particle.x.astype(cp.float32).get()[:,[0,2,1]], point_size=0.003)

plot.display()

Output()

# Simulation

In [11]:
import numpy as np
from tqdm.notebook import tqdm
from timeit import default_timer as timer
from torch.utils.dlpack import to_dlpack
import torch
import torch.nn as nn
from model_3d import UNet

g_size = tuple(GRES.tolist())
stg_size = tuple((GRES*2+1).tolist())
data_size = (112,176,112,1) # nearest value could be divisible by 16

x_pad_l = int((data_size[0]  - stg_size[0]) / 2)
y_pad_l = int((data_size[1]  - stg_size[1]) / 2)
z_pad_l = int((data_size[2]  - stg_size[2]) / 2)

vx_sympad, vy_sympad, vz_sympad, lvol_sympad, sphi_sympad, dxdx, dxdy, dxdz, dydx, dydy, dydz, dzdx, dzdy, dzdz = cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size),\
cp.ones(data_size)*-1, cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size),\
cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size), cp.zeros(data_size)

def grad_v(gx_v, gy_v, gz_v, dxdx, dxdy, dxdz, dydx, dydy, dydz, dzdx, dzdy, dzdz):
    
    # dxdx at center
    dxdx[1:-1, :, :,:] = gx_v[0:-2, :,:, :] - gx_v[2:, :,:, :]
    dxdx[1:-1, :,:, :][gx_v[0:-2, :,:, :]==0] = 0
    dxdx[1:-1, :, :,:][gx_v[2:, :,:, :]==0] = 0
    # dxdy at corner
    dxdy[:, 1:-1, :,:] = gx_v[:, 0:-2,:, :] - gx_v[:, 2:,:, :]
    dxdy[:, 1:-1, :,:][gx_v[:, 0:-2,:, :] ==0] = 0
    dxdy[:, 1:-1, :,:][gx_v[:, 2:,:, :] ==0] = 0
    #dxdz at corner
    dxdz[:,:,1:-1,:] = gx_v[:,:,0:-2,:] - gx_v[:,:,2:,:]
    dxdz[:,:,1:-1,:][gx_v[:,:,0:-2,:]==0] = 0
    dxdz[:,:,1:-1,:][gx_v[:,:,2:,:]==0] = 0

    # dydx at corner
    dydx[1:-1, :, :,:] = gy_v[0:-2, :,:, :] - gy_v[2:, :,:, :]
    dydx[1:-1, :,:, :][gy_v[0:-2, :, :,:] ==0] = 0
    dydx[1:-1, :, :,:][gy_v[2:, :,:, :] ==0] = 0        
    # dydy at center
    dydy[:, 1:-1, :,:] = gy_v[:, 0:-2,:, :] - gy_v[:, 2:,:, :]
    dydy[:, 1:-1, :,:][gy_v[:, 0:-2,:, :] ==0] = 0
    dydy[:, 1:-1, :,:][gy_v[:, 2:, :,:] ==0] = 0
    #dydz at corner
    dydz[:,:,1:-1,:] = gy_v[:,:,0:-2,:] - gy_v[:,:,2:,:]
    dydz[:,:,1:-1,:][gy_v[:,:,0:-2,:]==0] = 0
    dydz[:,:,1:-1,:][gy_v[:,:,2:,:]==0] = 0

    #dzdx at corner
    dzdx[1:-1, :, :,:] = gz_v[0:-2, :,:, :] - gz_v[2:, :,:, :]
    dzdx[1:-1, :,:, :][gz_v[0:-2, :, :,:] ==0] = 0
    dzdx[1:-1, :, :,:][gz_v[2:, :,:, :] ==0] = 0
    #dzdy at corner
    dzdy[:, 1:-1, :,:] = gz_v[:, 0:-2,:, :] - gz_v[:, 2:,:, :]
    dzdy[:, 1:-1, :,:][gz_v[:, 0:-2,:, :]==0] = 0
    dzdy[:, 1:-1, :,:][gz_v[:, 2:,:, :] ==0] = 0
    # dzdz at center
    dzdz[:,:, 1:-1,:] = gz_v[:,:, 0:-2, :] - gz_v[:,:, 2:, :]
    dzdz[:,:, 1:-1,:][gz_v[:,:, 0:-2, :]==0] = 0
    dzdz[:,:, 1:-1,:][gz_v[:,:, 2: , :] ==0] = 0
    

def unet_solve(vx, vy, vz, sphi, lvol, vx_sympad, vy_sympad, vz_sympad, lvol_sympad, sphi_sympad, dxdx, dxdy, dxdz, dydx, dydy, dydz, dzdx, dzdy, dzdz):
    vx_sympad[x_pad_l:x_pad_l+stg_size[0]:2, y_pad_l+1:y_pad_l+stg_size[1]:2, z_pad_l+1:z_pad_l+stg_size[2]:2,0] = vx
    vy_sympad[x_pad_l+1:x_pad_l+stg_size[0]:2, y_pad_l:y_pad_l+stg_size[1]:2, z_pad_l+1:z_pad_l+stg_size[2]:2,0] = vy
    vz_sympad[x_pad_l+1:x_pad_l+stg_size[0]:2, y_pad_l+1:y_pad_l+stg_size[1]:2, z_pad_l:z_pad_l+stg_size[2]:2,0] = vz
    sphi_sympad[x_pad_l:x_pad_l+stg_size[0], y_pad_l:y_pad_l+stg_size[1], z_pad_l:z_pad_l+stg_size[2],0] = sphi
    lvol_sympad[x_pad_l:x_pad_l+stg_size[0], y_pad_l:y_pad_l+stg_size[1], z_pad_l:z_pad_l+stg_size[2],0] = lvol/(0.0125**3)
    
    grad_v(vx_sympad, vy_sympad, vz_sympad, dxdx, dxdy, dxdz, dydx, dydy, dydz, dzdx, dzdy, dzdz)
    
    sphi_sympad[sphi_sympad>0]=2
    sphi_sympad[sphi_sympad<=0]=1
    sphi_sympad[sphi_sympad==2]=0
    
    input = cp.concatenate([dxdx, dydy, dzdz, dxdy, dxdz, dydx, dydz, dzdx, dzdy, sphi_sympad, lvol_sympad], axis=3)
    input = input.transpose((3, 0, 1, 2)).astype(np.float32)
    input = cp.expand_dims(input, axis=0)
    cuda.synchronize()

    model = UNet(in_channels=input.shape[1])
    model.load_state_dict(torch.load(ckpt_file)['net'])
    model.to(f"cuda:{device_num}")
    output = cp.from_dlpack(to_dlpack(model(torch.torch.as_tensor(input, device=f'cuda:{device_num}'))))/int(1/DT)
    
    delvx = output[0,0,x_pad_l:x_pad_l+stg_size[0]:2, y_pad_l+1:y_pad_l+stg_size[1]:2, z_pad_l+1:z_pad_l+stg_size[2]:2]
    delvy = output[0,1,x_pad_l+1:x_pad_l+stg_size[0]:2, y_pad_l:y_pad_l+stg_size[1]:2, z_pad_l+1:z_pad_l+stg_size[2]:2]
    delvz = output[0,2,x_pad_l+1:x_pad_l+stg_size[0]:2, y_pad_l+1:y_pad_l+stg_size[1]:2, z_pad_l:z_pad_l+stg_size[2]:2] 

    return delvx, delvy, delvz

In [12]:
print(f'the solver is {solver}!')
from tqdm.notebook import tqdm
import pickle
particle_series = {0.0: particle.x.copy().astype(cp.float32).get()[:,[0,2,1]]}
sphi_series = {0.0: solid_levelset.phi.copy().astype(cp.float32).get().transpose(1,2,0)}

ml_data={}

DURATION =  3
iterations = 0
current_time = 0.0
pbar = tqdm(total=DURATION)


p2g_all =0
visco_all=0
press_all=0
g2p_all = 0

while current_time < DURATION:
    if solver == 'unet':
        current_dt = DT
    elif solver == 'apic':
        cfl_dt = GDX / max(1e-10, cp.max(cp.sum(particle.v ** 2, axis=-1) ** 0.5).item())
        current_dt = min(DT, cfl_dt, DURATION - current_time)    
    current_time += current_dt
    pbar.update(current_dt)
    pbar.set_description(f"{current_dt = }")
    
    old_x = particle.x.copy()
    particle.x += particle.v * current_dt
    new_x = particle.x.copy()
    sdf.project(rb_d, particle.x)
    
    ################# Density Solver #################
    compute_fluid_levelset(particle, fluid_levelset, GDX)
    compute_fluid_volume(particle, fluid_volume, particle.vol)
    
    DensitySolver.solve(RHO, current_dt, particle.x, particle.m, particle.vol, grid.x.v, grid.y.v, grid.z.v, solid_levelset.phi, solid_levelset.v, fluid_levelset.phi, fluid_volume.vol)

    
    compute_fluid_levelset(particle, fluid_levelset, GDX)
    compute_fluid_volume(particle, fluid_volume, particle.vol)
    
    ################# Particles to Grid #################
    grid.x.m *= 0
    grid.x.v *= 0
    grid.y.m *= 0
    grid.y.v *= 0
    grid.z.m *= 0
    grid.z.v *= 0
    p2gt=timer()
    p2g(particle, grid)
    print(f"p2g:{timer()-p2gt}")
    p2g_all+=(timer()-p2gt)
    ################# External Force #################
    grid.y.v += -10 * current_dt # gravity        
    
    ################# Viscosity Solver #################
    if MU>0:
        if solver=='apic':

            ml_data['vx']=cp.asnumpy(grid.x.v)
            ml_data['vy']=cp.asnumpy(grid.y.v)
            ml_data['vz']=cp.asnumpy(grid.z.v)                
            ml_data['sphi']=cp.asnumpy(solid_levelset.phi)
            ml_data['lvol']=cp.asnumpy(fluid_levelset.phi)
            ml_data['lphi']=cp.asnumpy(fluid_volume.vol)
            ml_data['mu']=MU
            ml_data['dt']=current_dt
            viscot = timer()
            ViscositySolver.solve(current_dt, MU, RHO, grid.x.v, grid.y.v, grid.z.v, solid_levelset.phi, solid_levelset.v, fluid_levelset.phi, fluid_volume.vol)
            cuda.synchronize()
            print(f"visco:{timer()-viscot}")
            visco_all+=(timer()-viscot)

            ml_data['vx_new']=cp.asnumpy(grid.x.v)
            ml_data['vy_new']=cp.asnumpy(grid.y.v)
            ml_data['vz_new']=cp.asnumpy(grid.z.v)
            
        elif solver =='unet':
            viscot = timer()
            delvx, delvy, delvz = unet_solve(grid.x.v, grid.y.v, grid.z.v, solid_levelset.phi, fluid_volume.vol, vx_sympad, vy_sympad, vz_sympad, lvol_sympad, sphi_sympad, dxdx, dxdy, dxdz, dydx, dydy, dydz, dzdx, dzdy, dzdz)
            grid.x.v+=delvx
            grid.y.v+=delvy
            grid.z.v+=delvz
            grid.x.v[grid.x.m==0]=0
            grid.y.v[grid.y.m==0]=0
            grid.z.v[grid.z.m==0]=0
            print(f"visco:{timer()-viscot}")
            visco_all+=(timer()-viscot)

    
    ################# Pressure Solver #################
    
    presst= timer()
    PressureSolver.solve(grid.x.v, grid.y.v, grid.z.v, solid_levelset.phi, solid_levelset.v, fluid_levelset.phi, wx=DensitySolver.wx, wy=DensitySolver.wy, wz=DensitySolver.wz)
    print(f"press:{timer()-presst}")
    press_all+=(timer()-presst)

    extrapolate(GRES, 2, grid.x.v, grid.y.v, grid.z.v, grid.x.m, grid.y.m, grid.z.m)
    
    ################# Boundary Condition #################
    apply_boundary_condition(grid, solid_levelset, GDX)
     

    ################# Grid to Particles #################
    g2pt = timer()
    g2p(particle, grid)
    cuda.synchronize()
    g2p_all+=(timer()-g2pt)

    iterations += 1
    if (iterations % int(1/DT/20)==0):
        particle_series[current_time] = particle.x.get().astype(cp.float32)[:,[0,2,1]]
    print(p2g_all, g2p_all, visco_all, press_all)

with open(f'./ps.pickle', 'wb') as f:
    pickle.dump(particle_series, f)

the solver is unet!


  0%|          | 0/3 [00:00<?, ?it/s]

p2g:0.2969651259481907
visco:2.3719967361539602
press:1.5048033827915788
0.2970319176092744 0.23372141364961863 2.372052827849984 1.504862044006586
p2g:0.00623371172696352
visco:0.9032554905861616
press:0.7880311952903867
0.30329637974500656 0.23824234958738089 3.2753701293841004 2.292921779677272
p2g:0.006226913072168827
visco:0.859590851701796
press:0.7050850400701165
0.3095498140901327 0.2427525157108903 4.135021622292697 2.998036711476743
p2g:0.0062112221494317055
visco:0.8459461471065879
press:0.8146930811926723
0.3157891370356083 0.2472758525982499 4.981032979674637 3.812761983834207
p2g:0.006157601252198219
visco:0.8474321234971285
press:0.8067073859274387
0.3219737382605672 0.2517485087737441 5.8285217536613345 4.619500250555575
p2g:0.006162052042782307
visco:0.8461140403524041
press:0.7692948216572404
0.32816359121352434 0.25622478406876326 6.6746947560459375 5.388826632872224
p2g:0.006172552704811096
visco:0.8558247517794371
press:0.740844689309597
0.3343619033694267 0.260731

In [13]:
### from matplotlib import pyplot as plt
import k3d
import cupy as cp
%matplotlib widget

import pickle

K3D_BOUND = []
K3D_BOUND.append(BOUND_MIN[[0,2,1]].get())
K3D_BOUND.append((BOUND_MIN+BOUND_SIZE)[[0,2,1]].get())
ps = {}
count = 0

for key in particle_series.keys():
    ps[key] = particle_series[key][0:-1:1,:]
    count+=1

plot = k3d.plot(grid=[*K3D_BOUND[0], *K3D_BOUND[1]], grid_auto_fit=False)
plot += k3d.points(particle_series, point_size=0.03, opacity=1)
plot += k3d.marching_cubes(sphi_series[0.0], level=0, opacity=0.2, color=0xFF0000, bounds=[K3D_BOUND[i%2][i//2] for i in range(6)])

# plot += k3d.points(particle.x.astype(cp.float32).get()[:,[0,2,1]], point_size=0.03)
# plot += k3d.vectors(particle.x.astype(cp.float32).get(), particle.v.astype(cp.float32).get())
plot.display()
plot.start_auto_play()

Output()